In [1]:
import pandas as pd
import numpy as np
import nltk


In [15]:
data = open("data\\train.txt", 'r')

In [16]:
texts = []
emotions = []
for i in data:
    text, emotion = i.split(';')
    texts.append(text)
    emotions.append(emotion[:-1])

In [17]:
df = pd.concat([pd.Series(texts, name='Text'), pd.Series(emotions, name='Emotion')], axis=1)
df

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [18]:
X = df.iloc[:, 0]
y = df.iloc[:, -1]

In [19]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [21]:
ps = PorterStemmer()
wordnet = WordNetLemmatizer()
corpus = []

In [23]:
import re

In [25]:
for i in range(0, len(df)):
    review = re.sub('(^a-zA-Z)', ' ', df['Text'][i])
    review = review.lower()
    review = review.split()
    review = [wordnet.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus

['didnt feel humiliated',
 'go feeling hopeless damned hopeful around someone care awake',
 'im grabbing minute post feel greedy wrong',
 'ever feeling nostalgic fireplace know still property',
 'feeling grouchy',
 'ive feeling little burdened lately wasnt sure',
 'ive taking milligram time recommended amount ive fallen asleep lot faster also feel like funny',
 'feel confused life teenager jaded year old man',
 'petronas year feel petronas performed well made huge profit',
 'feel romantic',
 'feel like make suffering seeing mean something',
 'feel running divine experience expect type spiritual encounter',
 'think easiest time year feel dissatisfied',
 'feel low energy thirsty',
 'immense sympathy general point possible proto writer trying find time write corner life sign agent let alone publishing contract feel little precious',
 'feel reassured anxiety side',
 'didnt really feel embarrassed',
 'feel pretty pathetic time',
 'started feeling sentimental doll child began collection vint

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [35]:
len(X[1])

5000

In [37]:
yy = pd.get_dummies(y)

In [38]:
yy

,anger,fear,joy,love,sadness,surprise
0,False,False,False,False,True,False
1,False,False,False,False,True,False
2,True,False,False,False,False,False
3,False,False,False,True,False,False
4,True,False,False,False,False,False
...,...,...,...,...,...,...
15995,False,False,False,False,True,False
15996,False,False,False,False,True,False
15997,False,False,True,False,False,False
15998,True,False,False,False,False,False


In [39]:
y_final = pd.Series(y)
labels = y_final.unique()
mapping = {i:j for j,i  in enumerate(labels, 0)}
y_final = y_final.map(mapping)
y_final

0        0
1        0
2        1
3        2
4        1
        ..
15995    0
15996    0
15997    5
15998    1
15999    0
Name: Emotion, Length: 16000, dtype: int64

In [40]:
y_final = y_final.to_numpy()
y_final.shape

(16000,)

In [41]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y_final,test_size=0.20, random_state=0)

In [42]:
from sklearn.naive_bayes import MultinomialNB
emotion_detect_model = MultinomialNB().fit(X_train, y_train)

In [44]:
y_pred = emotion_detect_model.predict(X_test)

In [45]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_m = confusion_matrix(y_test, y_pred)
confusion_m

array([[825,  17,   8,   8,  13,  34],
       [ 64, 350,   6,   2,  13,  24],
       [ 28,   8, 144,   0,   5,  86],
       [ 18,   0,   1,  48,  17,  31],
       [ 38,  30,   2,   4, 298,  25],
       [ 41,  14,  38,   1,   7, 952]], dtype=int64)

In [49]:
from sklearn.metrics import accuracy_score

In [51]:
print(accuracy_score(y_pred, y_test))

0.8178125


In [53]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.885


In [54]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.823125


In [55]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.8728125


In [56]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.88625


c:\Users\Om Hemant Sonawane\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
